# Field boundary mapping in small scale farming using multi-resolution satellite data

The code was developed within a project funded by FAO, aiming at developing automatic workflows for mapping rice field boundaries in small-scale farming, using deep learning neural networks and advanced image processing. We conducted case studies in Cambodia and Vietnam, where rice paddy occupies a large portion of the agricultural area. We are facing two major challenges in the targeted research areas: (1) the lack of reference data, and (2) the fragmented agricultural areas characterized by very small fields (i.e., less than 1 ha). To overcome them, we used a convolutional deep learning U-Net network, able to achieve accurate segmentation result even with few training images.

In [33]:
import os
import numpy as np
import typing
import keras
import tensorflow as tf
from skimage import exposure
from osgeo import gdal

# Install keras-unet library for python
#%pip install git+https://github.com/karolzak/keras-unet


# Define dataset directory on Colab
# from google.colab import drive

BASE_PATH = "/home/jovyan/private/Agricultural_Field_Boundary"
INPUT_PATH = BASE_PATH + "/Training_dataset/"
OUTPUT_PATH = BASE_PATH + "/Networks/fine_tune_UNet5_Sentinel2/FAO_Cambodia_Unet"
EXPORT_OUTPUT_PATH = OUTPUT_PATH + "/Export_Path_Cambodia_SON_GM_2020"
IMAGE_PATH = "/Cambodia_SON_GM_2020"
LABEL_PATH = "/Classified"
PREDICTION_PATH = BASE_PATH + "/Training_dataset/" + "Cambodia" + "/Prediction_Cambodia_SON_GM_2020"

# INCLUDE_FOLDERS = ["Flevoland", "Friesland", "Gelderland", "Limburg", "Overijssel", "Zeeland", "Zuid-Holland", "Vietnam", "Cambodia"]
INCLUDE_FOLDERS = ["Cambodia"]
LEGEND = {
    1: 'Other',
    2: 'Field Boundary'
}


# To assess the accuracy you have to define the networks UUID and name here
NETWORK_UUID = "6c0f7e24-a85a-11ec-938c-02420a0001f1"

# Allowed Values:
#   * FCNDK3
#   * FCNDK4
#   * FCNDK5
#   * FCNDK6
#   * UNet2
#   * UNet3
#   * UNet5
NETWORK_NAME = "UNet5"


# To compile the model, also the optimizer has to be defined
NETWORK_OPTIMIZER = "Adam"

# Ensure you use the same optimizer parameters as in the training run
SGD_LEARNING_RATE = 0.0015
SGD_MOMENTUM = 0.9
ADAM_LEARNING_RATE = 0.001
ADAM_BETA_1 = 0.9
ADAM_BETA_2 = 0.999
ADAM_EPSILON = 1e-06

# Note: .ipynb_checkpoints are generated from the notebook interface
# when removing/adding image samples manually. They should be deleted,
# otherwise the Kernel will be died and forced to restart
for folder in INCLUDE_FOLDERS:
    original = INPUT_PATH + folder + IMAGE_PATH
    classified = INPUT_PATH + folder + LABEL_PATH
    if os.path.exists(f"{original}/.ipynb_checkpoints"):
        shutil.rmtree(f"{original}/.ipynb_checkpoints")
    if os.path.exists(f"{classified}/.ipynb_checkpoints"):
        shutil.rmtree(f"{classified}/.ipynb_checkpoints")



# Remark: All information in here could have been loaded from the training
#         configuration file as well. However, we missed the chance to export
#         the configuration as a machine readable type, e.g. JSON.
#         Thus, we did not pass the config from the readme manually but set the
#         parameters manually in here.


In [34]:
from tensorflow.python.keras import backend as K


In [35]:
# ################################################################
# Prerequisites for the network
#
# Includes a few helper functions which will be used to create and
# evaluate the network, the training and the accuracy.
# ################################################################

import imp, h5py
import pickle
import uuid
imp.reload(h5py)
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1.keras.backend import set_session

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.compat.v1.Session(config=config))

# ################################################################
# Helper functions - import/export of network
#
# These two functions help to export the network and import it.
# That allows to skip the training at a later stage.
# ################################################################

class ModelHistory:
    """Just a small container class to hold relevant information of a trained
    model.
    """

    def __init__(self, uuid, name, model, history, readme):
        """Create a new instance of this class
        :param uuid: A unique identifier of the network
        :param name: The networks name
        :param model: The pretrained model
        :param history: The training history of the model
        :param readme: A small readme with a summary of training parameters
        """
        self.uuid = uuid
        self.name = name
        self.model = model
        self.history = history
        self.readme = readme


def get_file_names(uuid, name, is_export=True):
    """Generates three file names for the model, weights and history file and
    the networks readme.

    File name order of returned tuple:
        * readme
        * model
        * weights
        * history

    :param uuid: Universal unique identifier of a trained network
    :param name: The networks name
    :return: Tuple with files in the order mentioned above
    """
    if is_export:
        base = f"{EXPORT_OUTPUT_PATH}/{str(uuid)}-{name}"
    else:
        base = f"{EXPORT_OUTPUT_PATH}/{str(uuid)}-{name}"


    f_readme = f"{base}-readme.txt"
    f_model = f"{base}-model.h5"
    f_weights = f"{base}-weights.h5"
    f_history = f"{base}-history"

    return (f_readme, f_model, f_weights, f_history)


def export_model(m: ModelHistory):
    """If a model is sufficiently trained, it can be exported. This allows to
    simply save the models state and the training history. Whenever one want to
    use the model the next time, the training can be skipped, since the trained
    model can just be imported from files.

    :param model_history: The trained model and history to be stored
    """
    f_readme, f_model, f_weights, f_history = get_file_names(m.uuid,m.name,True)

    # save readme
    with open(f_readme, 'w') as f:
        f.write(m.readme)
    print(f"Exported README: {f_readme}")

    # save models & weights
    m.model.save(f_model)
    print(f"Exported model: {f_model}")
    m.model.save_weights(f_weights)
    print(f"Exported weights: {f_weights}")

    # save history
    with open(f_history, "wb") as f:
        pickle.dump(m.history, f)
    print(f"Exported history: {f_history}")

def import_model(uuid, name):
    """Previously exported models can be imported with this funciton.
    :param uuid: The networks uuid
    :param name: The networks name
    :return: Instance of ModelHistory
    """
    f_readme, f_model, f_weights, f_history = get_file_names(uuid, name,False)

    # Load readme
    with open(f_readme, 'r') as f:
        readme = "".join(f.readlines())
    print(f"Imported README: {f_readme}")

    # Load model & weights
    # model = tf.keras.models.load_model(f_model)
    # Load model & weights, setting compile to 'False' when applying customer
    # defined loss functions
    model = tf.keras.models.load_model(f_model, compile=False)
    print(f"Imported model: {f_model}")
    model.load_weights(f_weights)
    print(f"Imported weights: {f_weights}")

    # Load history
    with open(f_history, 'rb') as f:
        history = pickle.load(f)
    print(f"Imported history: {f_history}")

    return ModelHistory(uuid, name, model, history, readme)


In [36]:
# ################################################################
# Loading input data
#
# Input data is loaded into two dictionaries: 
#
# images: contains the 4-band images. The values are loaded as ints.
# labels: contains 3D arrays in which each pixel is assigned with 
#         a label "1" = other and "2" = field boundary
#
# ################################################################


def key_generator(file_name):
    """Generates the key of a file based on the file name. The resulting key is
    a tuple of the province as string & the file number index as int,
    e.g. ("gelderland", 29)
    """
    file_name = file_name.lower()
    file_name = file_name.replace("classified_", "")
    file_name = file_name.replace("original_", "")
    file_name = file_name.replace(".tif", "")
    # TODO: Some images are named incorrectly
    #       (e.g. no '_' between the province name and the image index)
    (province, index) = tuple(file_name.split("_"))
    index = int(index)
    return (province, index)

def gtiff_to_array_geo(file_path):
    """Takes a file path and returns a tif file as a 3-dimensional numpy array, width x height x bands."""
    data = gdal.Open(file_path)
    bands = [data.GetRasterBand(i+1).ReadAsArray() for i in range(data.RasterCount)]
    return np.stack(bands, axis=2), data.GetGeoTransform(), data.GetProjection()

def gtiff_to_array(file_path):
    """Takes a file path and returns a tif file as a 3-dimensional numpy array, width x height x bands."""
    data = gdal.Open(file_path)
    bands = [data.GetRasterBand(i+1).ReadAsArray() for i in range(data.RasterCount)]
    return np.stack(bands, axis=2)


def transform_classification_image(input):
    """Takes the classification image input (in RGB format as 3D array) and 
    creates a 2D array out of this. The innermost array expects either values of
    [0, 0, 0] of [255, 255, 255] since this is the colouring we assigned to the
    classified images.

    :param input: 3D input image (classification)
    :return: 2D array image with labels 1 for 'other' and 2 for 'field_boundaries'
    """

    # Out of the 3D input array it takes the "max" element out of the array
    # This will either be 0 or 255. This function is just called to transform 
    # the 3D array to a 2D array.
    result = np.reshape(np.max(input, axis=2), (input.shape[0], input.shape[1], 1))

    # Now the array consists of pixels with values "0" or "255". We transform
    # each value, that is larger than 0 (i.e. 255) and assign the label "2" to
    # it. Each other element (i.e. 0) will get assigned the label "1".
    result = np.where(result > 0, 2, 1)
    return result

# Dictionaries which contain the input data
x_dict = {}
y_dict = {}
geoTrans = {}
geoProj = {}

# Iterate through defined folders and load all image data into the dictionaries
# image_data and label_data. Images can be accessed with (<province>, <index>)
for folder in INCLUDE_FOLDERS:
    original = INPUT_PATH + folder + IMAGE_PATH
    classified = INPUT_PATH + folder + LABEL_PATH
    for f in os.listdir(original): 
        key = key_generator(f)
        x_dict[key], geoTrans[key], geoProj[key] = gtiff_to_array_geo(original + "/" + f)

print(f"Total number of image & label tiles: {len(x_dict)}")

Total number of image & label tiles: 2


In [37]:
# ################################################################
# Normalizing images
#
# Normalizing all input images into values in the interval [0, 1].
# All bands are normalized seperately, which means, the min & max
# of each band is calculated based on each band of the image data.
# ################################################################

def normalize_array_1(arr):
    """Takes a 3D array as input, iterates over the bands and normalizes those.

    :param arr: input array (original image data) 
    :return: normalized data with values between 0 and 1
    """
    arr_norm = np.zeros(arr.shape, dtype=np.float32)

    for i in range(arr.shape[2]):
        min = arr[:, :, i].min()
        max = arr[:, :, i].max()
        arr_norm[:,:,i] = (arr[:,:,i] - min) / (max - min)
    return arr_norm

def normalize_array_2(arr, minimum, maximum):

    arr_norm = np.zeros(arr.shape, dtype=np.float32)

    for i in range(arr.shape[2]):
        arr_norm[:,:,i] = (arr[:,:,i] - minimum[i]) / (maximum[i] - minimum[i])
    return arr_norm

def normalize_array_3(arr, mean, sd):
    
    arr_norm = np.zeros(arr.shape, dtype=np.float32)

    for i in range(arr.shape[2]):
        arr_norm[:,:,i] = (arr[:,:,i] - mean[i]) / sd[i]
    return arr_norm

def get_feature_mins_maxs(images):
    """get the means and standard deviations per band for all image data
    :param image: list of image data
    :return: minima and maxima per band
    """
    features_mins = []
    features_maxs = []
    arr = np.array(images)
    for i in range(arr.shape[-1]):
        features_mins.append(np.min(arr[:, :, :, i]))
        features_maxs.append(np.max(arr[:, :, :, i]))
    return np.array(features_mins), np.array(features_maxs)

features_mins, features_maxs = get_feature_mins_maxs(list(x_dict.values()))
for k, v in x_dict.items():
    x_dict[k] = normalize_array_2(v, features_mins, features_maxs)
    # x_dict[k] = normalize_array_1(v)
    print(f"Performed normalization of {k[0]}_{k[1]}")


Performed normalization of cambodia_14
Performed normalization of cambodia_13


In [38]:
# ################################################################
# Network builder functions
#
# Dynamic builder function for the FCN-DK (supposted from layer 2 to 6)
# and unet (layer 1 to 5).
# ################################################################

#import model_segnet_Nbands

from keras.layers import Activation, BatchNormalization, Convolution2D, LeakyReLU, Reshape, ZeroPadding2D
from keras.models import Sequential
# TODO: Consider which optimizer is the best
from tensorflow.keras.optimizers import SGD, Adam


from keras_unet.models import satellite_unet

def build_unet(
    x: int,
    y: int,
    bands: int,
    labels: int,
    layers: int = 2,
) -> tf.keras.Model:
    """Create  a model of the popular U-Net network.

    :param x: Number of rows (x-shape)
    :param y: Number of columns (y-shape)
    :param bands: Number of bands (z-shape)
    :param lables: Number of labels to predict with the network
    :param layers: Number of layers of the network
    :return: Model of the corresponding U-Net network
    """
    model = satellite_unet(
        input_shape=(x, y, bands),
        num_classes=labels,
        output_activation="softmax",
        num_layers=layers,
    )
    return model

def build_unet2(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an UNet with 2 layers
    """
    return build_unet(x, y, bands, labels, layers=2)
    
def build_unet3(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an UNet with 3 layers
    """
    return build_unet(x, y, bands, labels, layers=3)

def build_unet5(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an UNet with 5 layers
    """
    return build_unet(x, y, bands, labels, layers=5)
    


def build_fcndk(
    x: int,
    y: int,
    prediction: int,
    labels: int,
    layers=4,
) -> tf.keras.Model:
    """Build a new network model based on the configuration of the networks 
    FCNDK2, ..., FCNDK6. Specify the layers to use in the parameters.

    :param x: Number of rows
    :param y: Number of columns
    :param bands: Number of bands in the input images
    :param labels: Number of different labels to choose as the classification
    :param layers: The number of FCNDK layers; Should be between 2 and 6 [default: 4]
    :return: Model of the corresponding FCNDK network
    """
    """Model builder function for FCN-DK6."""
    model = keras.models.Sequential()
    model.add(ZeroPadding2D((2, 2), input_shape=(x, y, bands)))
    model.add(Convolution2D(
              filters=16,
              kernel_size=(5, 5),
              dilation_rate=(1, 1)))
    model.add(BatchNormalization(axis=3))
    model.add(LeakyReLU(0.1))

    if layers >= 2:
        # FCNDK2
        model.add(ZeroPadding2D((4, 4)))
        model.add(Convolution2D(
                filters=32,
                kernel_size=(5, 5),
                dilation_rate=(2, 2)
        ))
        model.add(BatchNormalization(axis=3))
        model.add(LeakyReLU(0.1))

    if layers >= 3:
        # FCNDK3
        model.add(ZeroPadding2D((6, 6)))
        model.add(Convolution2D(
                filters=32,
                kernel_size=(5, 5),
                dilation_rate=(3, 3)
        ))
        model.add(BatchNormalization(axis=3))
        model.add(LeakyReLU(0.1))

    if layers >= 4:
        # FCNDK4
        model.add(ZeroPadding2D((8, 8)))
        model.add(Convolution2D(
                filters=32,
                kernel_size=(5, 5),
                dilation_rate=(4, 4)
        ))
        model.add(BatchNormalization(axis=3))
        model.add(LeakyReLU(0.1))

    if layers >= 5:
        # FCNDK5
        model.add(ZeroPadding2D((10, 10)))
        model.add(Convolution2D(
                filters=32,
                kernel_size=(5, 5),
                dilation_rate=(5, 5)
        ))
        model.add(BatchNormalization(axis=3))
        model.add(LeakyReLU(0.1))

    if layers >= 6:
        # FCNDK6
        model.add(ZeroPadding2D((12, 12)))
        model.add(Convolution2D(
                filters=32,
                kernel_size=(5, 5),
                dilation_rate=(6, 6)
        ))
        model.add(BatchNormalization(axis=3))
        model.add(LeakyReLU(0.1))

    # Output layer
    model.add(Convolution2D(
              filters=labels,
              kernel_size=(1, 1)
    ))

    model.add(keras.layers.Activation(
              activation="softmax"
    ))
    return model

def build_fcndk3(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an FCNDK with 3 layers
    """
    return build_fcndk(x, y, bands, labels, layers=3)
    
def build_fcndk4(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an FCNDK with 4 layers
    """
    return build_fcndk(x, y, bands, labels, layers=4)

def build_fcndk5(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an FCNDK with 5 layers
    """
    return build_fcndk(x, y, bands, labels, layers=5)

def build_fcndk6(
    x: int,
    y: int,
    bands: int,
    labels: int,
) -> tf.keras.Model:
    """Wrapper function to build an FCNDK with 6 layers
    """
    return build_fcndk(x, y, bands, labels, layers=6)


def build_network(name: str) -> typing.Callable:
    """Builds a new network, based on the networks name
    :param name: The networks name
    :return: The builder function of the corresponding network.
    """
    if name.lower() == "fcndk3":
        return build_fcndk3
    elif name.lower() == "fcndk4":
        return build_fcndk4
    elif name.lower() == "fcndk5":
        return build_fcndk5
    elif name.lower() == "fcndk6":
        return build_fcndk6
    elif name.lower() == "unet2":
        return build_unet2
    elif name.lower() == "unet3":
        return build_unet3
    elif name.lower() == "unet5":
        return build_unet5
    

In [39]:
# ################################################################
# Creating network
#
# In this code block, the network configuration is loaded properly
# and the corresponding builder function is called.
# ################################################################

NUMBER_BANDS = 4
NUMBER_CLASSES = 2
NUMBER_EPOCHS = 10
NUMBER_BATCHES = 64
VALIDATION_SPLIT = 0.02

model_builder = build_network(NETWORK_NAME)

# Optimizer (actually  not required anymore, but code legacy requires it.)
if NETWORK_OPTIMIZER == "Adam":
    OPTIMIZER = tf.keras.optimizers.Adam(
        learning_rate=ADAM_LEARNING_RATE,
        beta_1=ADAM_BETA_1, 
        beta_2=ADAM_BETA_2, 
        epsilon=ADAM_EPSILON
    )
elif NETWORK_OPTIMIZER == "SGD":
    OPTIMIZER = tf.keras.optimizers.SGD(
        learning_rate=SGD_LEARNING_RATE, 
        momentum=SGD_MOMENTUM
    )

# Load existing network from files
m = import_model(NETWORK_UUID, NETWORK_NAME)
readme = m.readme
model = m.model
history = m.history
print(f"Load existing network: {NETWORK_UUID} {NETWORK_NAME}")

# Print configuration README before (possible) training
print(readme)


Imported README: /home/jovyan/private/Agricultural_Field_Boundary/Networks/fine_tune_UNet5_Sentinel2/FAO_Cambodia_Unet/Export_Path_Cambodia_SON_GM_2020/6c0f7e24-a85a-11ec-938c-02420a0001f1-UNet5-readme.txt
Imported model: /home/jovyan/private/Agricultural_Field_Boundary/Networks/fine_tune_UNet5_Sentinel2/FAO_Cambodia_Unet/Export_Path_Cambodia_SON_GM_2020/6c0f7e24-a85a-11ec-938c-02420a0001f1-UNet5-model.h5
Imported weights: /home/jovyan/private/Agricultural_Field_Boundary/Networks/fine_tune_UNet5_Sentinel2/FAO_Cambodia_Unet/Export_Path_Cambodia_SON_GM_2020/6c0f7e24-a85a-11ec-938c-02420a0001f1-UNet5-weights.h5
Imported history: /home/jovyan/private/Agricultural_Field_Boundary/Networks/fine_tune_UNet5_Sentinel2/FAO_Cambodia_Unet/Export_Path_Cambodia_SON_GM_2020/6c0f7e24-a85a-11ec-938c-02420a0001f1-UNet5-history
Load existing network: 6c0f7e24-a85a-11ec-938c-02420a0001f1 UNet5

Training configuration
Network
    UUID:               6c0f7e24-a85a-11ec-938c-02420a0001f1
    Name:            

In [40]:
# ################################################################
# Predictions & export
#
# Here, the network is fed with the given input files. Resulting
# predictions are exported as TIF files.
# ################################################################

from matplotlib import pyplot
from PIL import Image

def evaluate_predictions(
    input: np.ndarray,
    nc: int,
    f_weights: str,
    optimizer: tf.keras.optimizers.Optimizer,
    model_builder: typing.Callable,
) -> np.ndarray:
    """Takes an input image, patches it into smaller patches and feeds the FCN
    with each of the patches. The output samples are patches of the predicted
    classification. These patches are combined into one large image, that can
    be compared with the classified image of the corresponding input data.

    :param input: test image to evaluate
    :param nc: Number of classes/labels
    :param f_weights: File path to the corresponding weights file
    :param optimizer: Optimizer for the network
    :param model_build: method to create the model
    :return: 2D ndarray of the predicted labels
    """
    x, y, bands = input.shape

    # Build model and load model weights
    model = model_builder(x, y, bands, nc)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
    #model.compile(optimizer=OPTIMIZER, loss=focal_tversky_loss, metrics='accuracy')
    model.load_weights(f_weights)

    # Predict field boundaries in network
    # Increase dimension to perform batch prediction
    input = np.expand_dims(input, 0)
    prediction = model.predict(input)[0]
    # Map highest score onto label
    # prediction = np.argmax(prediction, axis=2) + 1
    return prediction[:,:,1]

def export_array(labels: np.ndarray, filename: str):
    """Maps labels onto the colors black & white and exports the resulting image
    as a file with the given file.

    :param labels: 2D array of labels per pixel
    :param filename: File name of the new file
    """
    x, y = labels.shape

    img = np.zeros((x, y, 3), dtype=np.uint8)

    for i in range(img.shape[2]):
        img[:, :, i] = np.where(labels[:, :] == 2, 255, 0)

    Image.fromarray(img).save(filename)
    #Image.fromarray(img).save()
    #pyplot.imsave(filename, Image.fromarray(img))

def export_geotiff(labels: np.ndarray, geoTrans, geoProj, filename: str):
    """Maps labels onto the colors black & white and exports the resulting image
    as a file with the given file.

    :param labels: 2D array of labels per pixel
    :param filename: File name of the new file
    """
    x, y = labels.shape
    # labels[:, :] = np.where(labels[:, :] == 2, 65535, 0)
    labels[:, :] *= 65535
    # import matplotlib.pylab as plt
    # plt.hist(labels.reshape(x*y)); plt.show()
    driver = gdal.GetDriverByName("GTiff")
    outData = driver.Create(filename, x, y, 3, gdal.GDT_UInt16)

    # Write metadata
    outData.SetGeoTransform(geoTrans)
    outData.SetProjection(geoProj)

    # Write raster data sets
    for i in range(3):
      outData.GetRasterBand(i+1).WriteArray(labels)
    outData.FlushCache()
    outData = None     

In [41]:
# ################################################################
# Predict images and store as TIF
#
# This code block will iterate through the included folders and
# performs predictions on the satellite image tiles. The resulting
# images are mapped onto black & white and then exported as TIF
# files
# ################################################################

import pathlib


keys_per_province = dict()

for f in INCLUDE_FOLDERS:
    keys_per_province[f] = [k for k in x_dict.keys() if k[0].lower() == f.lower()]

for f, keys in keys_per_province.items():
    for k in keys:
        x = x_dict[k]

        (_, _, f_weights, _) = get_file_names(NETWORK_UUID, NETWORK_NAME)

        try:
            img = evaluate_predictions(
                x,
                NUMBER_CLASSES,
                f_weights,
                OPTIMIZER,
                model_builder,
            )
        except Exception as e:
            print(f"Failed to predict image of size {x.shape[0]}x{x.shape[1]}")
            continue

        # Build directory name
        fname = PREDICTION_PATH
        fname += f"{f}/"
        fname += f"Prediction_{NETWORK_NAME}/"

        # Create directory
        pathlib.Path(fname).mkdir(parents=True, exist_ok=True)

        # build filename
        fname += f"prediction_{k[0]}_{k[1]}"
        fname += ".tif"

        # export_array(img, fname)
        export_geotiff(img, geoTrans[k],geoProj[k],fname)
        print(f"Exported prediction {fname}")

2023-08-14 14:47:04.562394: W tensorflow/core/common_runtime/bfc_allocator.cc:360] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


Exported prediction /home/jovyan/private/Agricultural_Field_Boundary/Training_dataset/Cambodia/Prediction_Cambodia_SON_GM_2020Cambodia/Prediction_UNet5/prediction_cambodia_14.tif
1/1 [==============================] - 2s 2s/step
Exported prediction /home/jovyan/private/Agricultural_Field_Boundary/Training_dataset/Cambodia/Prediction_Cambodia_SON_GM_2020Cambodia/Prediction_UNet5/prediction_cambodia_13.tif


In [46]:
import os
import subprocess

# Define the patch size and input file path
patch_width = 256
patch_height = 256
input_file = "/home/jovyan/private/Agricultural_Field_Boundary/Training_dataset/Cambodia/Prediction_Cambodia_SON_GM_2020Cambodia/Prediction_UNet5/prediction_cambodia_13.tif"
output_directory = "output_patches"
os.makedirs(output_directory, exist_ok=True)

# Open the input raster dataset to get its width and height
dataset = gdal.Open(input_file)
if dataset is None:
    print("Failed to open the input raster dataset.")
    exit()

width = dataset.RasterXSize
height = dataset.RasterYSize

# Calculate the number of patches in each dimension
num_patches_horizontal = width // patch_width
num_patches_vertical = height // patch_height

# Create a loop to split the image into patches
for i in range(num_patches_vertical):
    for j in range(num_patches_horizontal):
        patch_name = os.path.join(output_directory, f"patch_{i}_{j}.tif")
        x_offset = j * patch_width
        y_offset = i * patch_height
        command = f"gdal_translate -srcwin {x_offset} {y_offset} {patch_width} {patch_height} -of GTiff {input_file} {patch_name}"
        subprocess.call(command, shell=True)

print("Patches saved successfully.")

Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 768, 768
0...10...20...30...40...50...60...70...80...90...100 - done.
Patches saved successfully.


In [ ]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from skimage.segmentation import felzenszwalb
from skimage.measure import find_contours
from skimage.measure import regionprops
from shapely.geometry import Polygon

# Define the input and output folder paths
input_folder = "/home/jovyan/private/FAO_ParisC/Code/Code_github/output_patches/"
output_folder = "/home/jovyan/private/FAO_ParisC/Code/Code_github/output_shapefiles/Cambodia_SON"
os.makedirs(output_folder, exist_ok=True)

# Iterate over the TIFF files in the input folder
tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]
for tif_file in tif_files:
    # Open the grayscale image
    tif_path = os.path.join(input_folder, tif_file)
    with rasterio.open(tif_path) as src:
        grayscale_image = src.read(1)
        transform = src.transform

    # Perform graph-based segmentation
    segments = felzenszwalb(grayscale_image, scale=50, sigma=0.1)

    # Create an empty GeoDataFrame for the polygons
    polygons_gdf = gpd.GeoDataFrame(columns=['segment_id', 'geometry'])

    # Set the maximum segment size threshold
    max_segment_size = 1000

    # Set the intensity threshold for background segments
    background_threshold = 100

    # Set the solidity threshold for irregular fragments
    solidity_threshold = 0.7

    # Iterate over the segments and create polygons
    for segment_id in np.unique(segments):
        # Create a mask for the current segment
        segment_mask = (segments == segment_id)

        # Calculate the size of the segment (number of pixels)
        segment_size = np.sum(segment_mask)

        # Skip segments above the maximum size threshold
        if segment_size > max_segment_size:
            continue

        # Extract the intensity values of the segment
        segment_intensities = grayscale_image[segment_mask]

        # Check if the segment is predominantly background based on intensity threshold
        if np.mean(segment_intensities) < background_threshold:
            continue

        # Find the contour of the segment mask
        contour = find_contours(segment_mask, level=0.5)[0]

        # Adjust the contour coordinates to match the image orientation and transformation
        contour_coords = []
        for coord in contour:
            x, y = rasterio.transform.xy(transform, coord[0], coord[1])
            contour_coords.append([x, y])

        # Create a Polygon from the contour coordinates
        polygon = Polygon(contour_coords)

        # Calculate the solidity of the segment
        props = regionprops(segment_mask.astype(int))[0]
        solidity = props.solidity

        # Check if the segment has irregular fragments based on solidity threshold
        if solidity < solidity_threshold:
            continue

        # Add polygon to the GeoDataFrame
        polygons_gdf = polygons_gdf.append({'segment_id': segment_id, 'geometry': polygon}, ignore_index=True)

    # Export polygons as a shapefile
    tif_file_name = os.path.splitext(tif_file)[0]
    output_shapefile = os.path.join(output_folder, f"{tif_file_name}.shp")
    polygons_gdf.to_file(output_shapefile)

print("Polygonization completed successfully.")
